<a href="https://colab.research.google.com/github/PrinceSajjadHussain/12.-TensforFlow-Object-Detection/blob/master/License_Plate_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries
- cv2 is opencv library which stands for open source computer vision. Opecv is a collection os algorithms which are used for computer vision.
- numpy is stands for Numerical Python. It is useful library for scitific calcualtion in python. It has multidimentional array object and lots of usefull methods for calculations.
- matplotlib.pyplot is used for ploting images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Download Dataset
- Data set for this project is downloaded from server
- unzip the download .zip file to get the .json file from it

In [ ]:
#download dataset zip file
!wget -O dataset.zip https://cc-uploads.s3.amazonaws.com/open_links/HumAIn+2019/vehicle-number-plate-detection+Datasets.zip

    
# #unzip the downloaded zip file    
!unzip dataset.zip    

--2019-08-26 09:02:40--  https://cc-uploads.s3.amazonaws.com/open_links/HumAIn+2019/vehicle-number-plate-detection+Datasets.zip
Resolving cc-uploads.s3.amazonaws.com (cc-uploads.s3.amazonaws.com)... 52.216.163.163
Connecting to cc-uploads.s3.amazonaws.com (cc-uploads.s3.amazonaws.com)|52.216.163.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20255 (20K) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  19.78K  --.-KB/s    in 0.03s   

2019-08-26 09:02:40 (599 KB/s) - ‘dataset.zip’ saved [20255/20255]

Archive:  dataset.zip
  inflating: Indian_Number_plates.json  


# Load data from dataset json file 
- Dataset Indian_Number_plates.json is consisting of nested json object. So I am using pandas json_normalize method to make a dataframe as your requirement

In [ ]:
import json
from pandas.io.json import json_normalize

data = list()
with open('Indian_Number_plates.json') as data_file:
    for line in data_file:
        data.append(json.loads(line))

df = json_normalize(data=data,record_path=['annotation'],meta=['content'])

print(df.shape)
df.head()

(245, 6)


,imageHeight,imageWidth,label,notes,points,content
0,466,806,[number_plate],,"[{'x': 0.7220843672456576, 'y': 0.587982832618...",http://com.dataturks.a96-i23.open.s3.amazonaws...
1,449,494,[number_plate],,"[{'x': 0.16194331983805668, 'y': 0.85077951002...",http://com.dataturks.a96-i23.open.s3.amazonaws...
2,314,500,[number_plate],,"[{'x': 0.082, 'y': 0.697452229299363}, {'x': 0...",http://com.dataturks.a96-i23.open.s3.amazonaws...
3,469,500,[number_plate],,"[{'x': 0.434, 'y': 0.6673773987206824}, {'x': ...",http://com.dataturks.a96-i23.open.s3.amazonaws...
4,563,1001,[number_plate],,"[{'x': 0.2087912087912088, 'y': 0.390763765541...",http://com.dataturks.a96-i23.open.s3.amazonaws...


# Download images from url and store it into the test_images directory
- By using the dataframe 'content' columns url, I am downloading the images from server and store them inside the 'test_images' directory with sequential filename
- Filenames are stored inside a list named image_list

In [ ]:
#install scikit image
!pip install scikit-image

In [ ]:
from skimage import io

!mkdir -p test_images

image_list =list()
for i in range(df.shape[0]):
    file_name = "image"+str(i)+".jpg"
    file_path = "test_images/"+file_name
    url =  df['content'][i]
    image_list.append(file_name)
    img = io.imread(url)
    cv2.imwrite(file_path,img)

# Add downloaded image list to the dataframe
- the image filename list is inserted to the dataframe

In [ ]:
df.insert(0,"imageName",image_list)
df.head()

,imageName,imageHeight,imageWidth,label,notes,points,content
0,image0.jpg,466,806,[number_plate],,"[{'x': 0.7220843672456576, 'y': 0.587982832618...",http://com.dataturks.a96-i23.open.s3.amazonaws...
1,image1.jpg,449,494,[number_plate],,"[{'x': 0.16194331983805668, 'y': 0.85077951002...",http://com.dataturks.a96-i23.open.s3.amazonaws...
2,image2.jpg,314,500,[number_plate],,"[{'x': 0.082, 'y': 0.697452229299363}, {'x': 0...",http://com.dataturks.a96-i23.open.s3.amazonaws...
3,image3.jpg,469,500,[number_plate],,"[{'x': 0.434, 'y': 0.6673773987206824}, {'x': ...",http://com.dataturks.a96-i23.open.s3.amazonaws...
4,image4.jpg,563,1001,[number_plate],,"[{'x': 0.2087912087912088, 'y': 0.390763765541...",http://com.dataturks.a96-i23.open.s3.amazonaws...


# OCR of the license plates
- pytesseract library in python is used for ocr detection here.
- from dataset we get the normalised quadinate of the license plate
- using height and width of the dataset, I calculate the actual points
- cropped image is generated using this corner point
- then this cropped images is gives to the input of the image_to_string method fo pytesseract library
- All the license plate detected result is stored into the license_plates list
- This detected result is them added to dataframe corrosponding to the image

In [ ]:
## install tesseract 
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,084 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
import pytesseract

license_plates = list()
for i in range(df.shape[0]):
    
    width = df['imageWidth'][i]
    height = df['imageHeight'][i]
    x1 = int(df['points'][i][0]['x']*width)
    y1 = int(df['points'][i][0]['y']*height)
    x2 = int(df['points'][i][1]['x']*width)
    y2 = int(df['points'][i][1]['y']*height)
    
    img = cv2.imread("test_images/"+df['imageName'][i],0)
    cropped_img = img[y1:y2,x1:x2]
    
    license_plate_number =  pytesseract.image_to_string(cropped_img)
    license_plate_number = "".join([i for i in license_plate_number if i==' ' or i.isalnum()])
    license_plates.append(license_plate_number)
    
df.insert(0,"license_plate_number",license_plates)
df[df['license_plate_number'].str.len() > 7].head()

,license_plate_number,imageName,imageHeight,imageWidth,label,notes,points,content
3,KA03 1351,image3.jpg,469,500,[number_plate],,"[{'x': 0.434, 'y': 0.6673773987206824}, {'x': ...",http://com.dataturks.a96-i23.open.s3.amazonaws...
8,THO7BU5427,image8.jpg,1119,1569,[number_plate],,"[{'x': 0.41075794621026895, 'y': 0.45371428571...",http://com.dataturks.a96-i23.open.s3.amazonaws...
10,KL KLeSR 2473 ye,image10.jpg,400,642,[number_plate],,"[{'x': 0.3115264797507788, 'y': 0.345}, {'x': ...",http://com.dataturks.a96-i23.open.s3.amazonaws...
12,MH20CSi 1,image12.jpg,435,640,[number_plate],,"[{'x': 0.3359375, 'y': 0.42758620689655175}, {...",http://com.dataturks.a96-i23.open.s3.amazonaws...
15,HDL 7C N 3617,image15.jpg,642,774,[number_plate],,"[{'x': 0.35400516795865633, 'y': 0.53582554517...",http://com.dataturks.a96-i23.open.s3.amazonaws...


# Finally the result is stored into .csv file

In [ ]:
df.to_csv("result.csv")

# Referances
- https://pypi.org/project/pytesseract/
- https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_setup/py_intro/py_intro.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html
- https://scikit-image.org/docs/dev/api/skimage.io.html